In [ ]:
%pip install -U opensearch-py
%pip install -U boto3
%pip install -U retrying

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import boto3
import pprint
import random
import json

suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
response = sts_client.get_caller_identity()
print(json.dumps(response))
boto3_session = boto3.session.Session()
region = boto3_session.region_name
print(region)
pp = pprint.PrettyPrinter(indent=2)
account_id=sts_client.get_caller_identity().get('Account')
database_name="snowflake"

In [ ]:
import json

glue_client=boto3.client("glue")
response = glue_client.get_tables(
    CatalogId=account_id,
    DatabaseName=database_name,
)
snowflake_tables=response["TableList"]
print(json.dumps(snowflake_tables, indent=4, sort_keys=True, default=str))

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import time
service="aoss"
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

aoss_node="mey2dta082iatb0w5x43.us-east-1.aoss.amazonaws.com"
index_name = "snowflake"
body_json = {
    "settings": {
        "index.knn": "true",
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "name": "hnsw",
                    "engine": "faiss"
                },
            },
            "text": {
                "type": "text"
            },
            "text-metadata": {
                "type": "text"         }
        }
    }
}
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': aoss_node, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# Create index
response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))

print('\nCreating index:')
print(response)
time.sleep(5) # index creation can take up to a minute